In [2]:
!apt-get install -y tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (5,241 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [12]:
import zipfile

#Import a bunch of useful libraries
from PIL import Image
import os
import csv
import pytesseract
import re
import os
import time
import torch.optim as optim
from torch.autograd import Variable
from google.colab import files
from collections import Counter
from datetime import datetime

import argparse
import math
import time

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as npr
import scipy.misc
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import logging

import json
import pandas as pd

In [4]:
#Tesseract path
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

In [5]:
zip_path = '/content/valid_receipts_zip.zip'

if zipfile.is_zipfile(zip_path):
    print("The file is a valid ZIP.")
else:
    print("The file is not a valid ZIP (possibly corrupted or wrong format).")

assert zipfile.is_zipfile(zip_path)

The file is a valid ZIP.


In [6]:

extract_folder = '/content/valid_receipts'
os.makedirs(extract_folder, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

In [20]:
receipts_folder = "/content/valid_receipts/valid_receipts"
output_csv = "extracted_receipts.csv"
cleaned_csv = "cleaned_receipts.csv"

In [8]:
def extract_text_from_image(image_path):
    """Extract text from an image using Tesseract OCR."""
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

In [9]:
def parse_receipt_text(text):
    """Extract date, items, and prices from receipt text using regex."""
    # Extract date (handles formats like 10/02/14 or 2024-03-09)
    date_patterns = [
        r'\b(\d{2})[\/\-](\d{2})[\/\-](\d{2,4})\b',  # Matches DD/MM/YY or MM/DD/YY
        r'\b(\d{4})[\/\-](\d{2})[\/\-](\d{2})\b'  # Matches YYYY-MM-DD
    ]
    date = "Unknown"

    for pattern in date_patterns:
        match = re.search(pattern, text)
        if match:
            groups = match.groups()
            if len(groups) == 3:
                day, month, year = groups
                if len(year) == 2:
                    year = "20" + year  # Assuming receipts are from 2000 onwards
                try:
                    date = datetime.strptime(f"{year}-{month}-{day}", "%Y-%m-%d").strftime("%Y-%m-%d")
                    break
                except ValueError:
                    continue

    # Extract item names and prices
    item_price_pattern = r'([A-Za-z0-9 &]+)\s+(\d+\.\d{2})'
    matches = re.findall(item_price_pattern, text)

    structured_data = []
    for match in matches:
        item, price = match
        structured_data.append({"date": date, "item": item.strip(), "price": price})

    return structured_data

In [10]:
def process_receipts():
    """Extracts and saves receipt details in a CSV file."""
    with open(output_csv, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["date", "item", "price"])

        i = 1

        for filename in os.listdir(receipts_folder):
            print('Processing receipt: ' + str(i))
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.pdf')):
                image_path = os.path.join(receipts_folder, filename)
                text = extract_text_from_image(image_path)
                structured_data = parse_receipt_text(text)

                i += 1

                for entry in structured_data:
                    writer.writerow([entry["date"], entry["item"], entry["price"]])
    print(f"Extraction completed. Data saved to {output_csv}")

In [11]:
process_receipts()

Processing receipt: 1
Processing receipt: 2
Processing receipt: 3
Processing receipt: 4
Processing receipt: 5
Processing receipt: 6
Processing receipt: 7
Processing receipt: 8
Processing receipt: 9
Processing receipt: 10
Processing receipt: 11
Processing receipt: 12
Processing receipt: 13
Processing receipt: 14
Processing receipt: 15
Processing receipt: 16
Processing receipt: 17
Processing receipt: 18
Processing receipt: 19
Processing receipt: 20
Processing receipt: 21
Processing receipt: 22
Processing receipt: 23
Processing receipt: 24
Processing receipt: 25
Processing receipt: 26
Processing receipt: 27
Processing receipt: 28
Processing receipt: 29
Processing receipt: 30
Processing receipt: 31
Processing receipt: 32
Processing receipt: 33
Processing receipt: 34
Processing receipt: 35
Processing receipt: 36
Processing receipt: 37
Processing receipt: 38
Processing receipt: 39
Processing receipt: 40
Processing receipt: 41
Processing receipt: 42
Processing receipt: 43
Processing receipt: 

In [29]:


# List of forbidden words
forbidden_words = ['Total', 'TAX', 'SUBTOTAL', 'BTOTAL', 'TOTAL', 'subtotal', 'total', 'Tax', 'tax', "", 'TOTAL TAX', 'TOTA', 'CASH', 'Debit Tend', 'Debit', 'Please Come', 'Transaction', 'Change Due', 'CAHNGE', 'Cashless', 'TRANSACTION AMOUNT', 'STOTAL', 'PAYMENT', 'SALES TAX', 'SALES TAX AMOUNT', 'VISA', 'DEBIT TEND', 'TAX 1', 'CHANGE CASH']

# Convert forbidden words to lowercase for case-insensitive comparison
forbidden_words = [word.lower() for word in forbidden_words]

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(output_csv)

# Check the structure of the DataFrame to ensure it's loaded correctly
print(df.head())

# Filter out rows where the 'item' column contains any forbidden word
# We also handle empty or NaN values in 'item' column
df_cleaned = df[~df['item'].apply(lambda x: any(word in str(x).lower() for word in forbidden_words) if pd.notnull(x) else False)]

# Check if rows are being dropped by checking the shape before and after filtering
print(f"Original DataFrame shape: {df.shape}")
print(f"Filtered DataFrame shape: {df_cleaned.shape}")

# Save the cleaned DataFrame to a new CSV file
df_cleaned.to_csv('cleaned_csv.csv', index=False)

# Print the cleaned DataFrame (optional)
print(df_cleaned.head())


      date                      item  price
0  Unknown                xKSWTR4OPK   6.79
1  Unknown  F 9600000000 HI BEVERAGE   2.40
2  Unknown      1356881 YOUTH SNORKE  29.99
3  Unknown           2622016 SNORKEL  36.99
4  Unknown        547591 HH TIARE 12  36.99
Original DataFrame shape: (268, 3)
Filtered DataFrame shape: (1, 3)
       date item  price
33  Unknown  NaN    5.0


In [40]:
#Data cleaning
forbidden_words = ['Total', 'TAX', 'SUBTOTAL', 'DEBIT PAY FROM PRIMARY', '1767 DEBIT TOTAL PAYMEN' , 'DEBIT LOAD', 'BTOTAL', 'TOTAL', 'subtotal', 'total', 'Tax', 'tax', "", 'TOTAL TAX', 'TOTA', 'CASH', 'Debit Tend', 'Debit', 'Please Come', 'Transaction', 'Cash Rounding' ,'Change Due', 'CHANGE', 'Cashless', 'TRANSACTION AMOUNT', 'STOTAL', 'PAYMENT', 'SALES TAX', 'SALES TAX AMOUNT', 'VISA', 'DEBIT TEND', 'TAX 1', 'CHANGE CASH', 'Please Come Again' , 'DEBIT et' , '	EFT DEBIT PAY FROM PRIMARY']

forbidden_words_lower = [word.lower() for word in forbidden_words]
print(forbidden_words_lower)

df = pd.read_csv(output_csv)

for index, row in df.iterrows():
  line = str(row['item'])
  line = line.lower()

  if line in forbidden_words_lower:
    df.drop(index, inplace=True)

df.to_csv('cleaned_csv.csv', index=False)

['total', 'tax', 'subtotal', 'debit pay from primary', '1767 debit total paymen', 'debit load', 'btotal', 'total', 'subtotal', 'total', 'tax', 'tax', '', 'total tax', 'tota', 'cash', 'debit tend', 'debit', 'please come', 'transaction', 'cash rounding', 'change due', 'change', 'cashless', 'transaction amount', 'stotal', 'payment', 'sales tax', 'sales tax amount', 'visa', 'debit tend', 'tax 1', 'change cash', 'please come again', 'debit et', '\teft debit pay from primary']
